### Measuring transaction costs and trading volume

In [60]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as scipy
sys.path.append("utils")
import dynamic_backtesting as db
from compare_strategies import calc_transaction_costs as ctc
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["black", "green",  "deepskyblue", "orange","grey", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [61]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

In [62]:
models = ['sGARCH10', 'sGARCH11', 'gjrGARCH11']
tuning_gamma_Ds = [1.52e-6, 1.3e-5, 1e-4]

In [63]:
def remove_extreme(num, tolerance):
    if num > tolerance:
        return tolerance
    
    return num

In [64]:
def calc_TC(model, tuning_gamma_D, assets):
    v_t_s, out_of_sample, in_sample, Omega_ts = db.garch_with_trading_cost(tickers=assets, model_type=model, tuning_gamma_D=tuning_gamma_D)
    out_of_sample = out_of_sample.divide(100)
    TC = ctc(v_t_s, out_of_sample, Omega_ts)
    TC = [remove_extreme(x, 1) for x in TC]
    return TC

In [65]:
def calc_multiple_TC(tickers):
    TCs = []
    for i, model in enumerate(models):
        row = []
        for j, tuning_gamma_D in enumerate(tuning_gamma_Ds):
            row.append(calc_TC(model, tuning_gamma_D, tickers))
        TCs.append(row)
    TCs_mean = [[np.nanmean(TC) for TC in TC_row] for TC_row in TCs]
    return TCs, TCs_mean

In [69]:
TC = ctc(v_t_s, out_of_sample, Omega_ts)

In [73]:
TC = [remove_extreme(x, 1) for x in TC]

In [77]:
[remove_extreme(x, 1) for x in TCs_all[0][0]]

[0.0,
 0.00021674743055229064,
 0.0018642609665630637,
 0.001385161000776346,
 0.0006157409256214657,
 0.0002989000446501271,
 0.0001785970761353111,
 0.00022413903459908325,
 0.0007500584449738874,
 0.0012111411833482843,
 0.0005595533828553175,
 0.004174607254365862,
 0.003205273659625826,
 0.03201898185046499,
 0.02593036681542178,
 0.0010155168822185603,
 0.00041131651247480636,
 0.00021310879391213609,
 0.0005653533518823707,
 0.0012010873358397393,
 0.006085720777483746,
 0.0030729296932557517,
 0.003980385769500868,
 0.010878904887363249,
 0.0018346761323436165,
 0.008823513318553146,
 0.03643557775427749,
 0.02935617630249408,
 0.020780767829351683,
 0.00020971654346195367,
 0.0034428226254217794,
 0.0035413114934468313,
 0.00033954246819648936,
 0.003117742453884913,
 0.007713294641237376,
 0.0448066807513044,
 0.026276168633914153,
 0.002012022144486985,
 0.009001230226138544,
 0.013104744694810471,
 0.00043617143276501936,
 0.0014308199027138436,
 0.0005425518482705949,
 0.0

In [66]:
TCs_all, TCs_all_mean = calc_multiple_TC(tickers)

['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.52e-06
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.3e-05
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.0001
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.52e-06
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.3e-05
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.5

In [ ]:
TCs_bonds, TCs_bonds_mean = calc_multiple_TC(bonds)

In [38]:
TCs_stocks, TCs_stocks_mean = calc_multiple_TC(stocks)

In [20]:
TCs_com, TCs_com_mean = calc_multiple_TC(commodities)

In [20]:
fig, ax = plt.subplots(1, 1, figsize=(7, 6), dpi=192)
ax.plot(TCs_all_mean)